In [ ]:
import cv2
import copy
import math
import os
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model
from keras.models import Sequential
from os import listdir
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
# -------------------------------------------------
# Constants
SIZE_FILTER_X = 36
SIZE_FILTER_Y = 36
# Parameters
NB_ITERATION = 20
THRESHOLD_MODEL = 0.005

In [ ]:
##
# Load keras model.
##
def load_model_keras(path_to_model):
    return load_model(path_to_model)

In [ ]:
##
# Return means positions of rectangles (list_point) which are 
# spatially close.
##
def compute_mean_rectangle(list_point, step_x, step_y):
    has_changed = True
    while(has_changed):
        has_changed = False
        temp_result = []
        list_indice = []
        for i in range(len(list_point)):
            Added_new_element = False
            for j in range(i+1, len(list_point)):
                if (math.fabs(list_point[i][0] - list_point[j][0]) <= 3*step_x and
                    math.fabs(list_point[i][1] - list_point[j][1]) <= 3*step_y and j not in list_indice):
                    
                    temp_result.append((int((list_point[i][0] + list_point[j][0])/2), int((list_point[i][1] + list_point[j][1])/2)))
                    list_indice.append(j)
                    Added_new_element = True
                    
            if Added_new_element == False and i not in list_indice:
                temp_result.append(list_point[i])
                
            has_changed = has_changed | Added_new_element
            
        list_point = copy.deepcopy(temp_result)
    
    return list_point
        

In [ ]:
##
# Search and return rectangles (list_point) with the highest scores locally.
##
def compute_highest_score(list_point, list_score, step_x, step_y, mode):
    
    ## Mode
    # 1 : Locally
    # 2 : Highest score on the entire image.
    
    if(mode == 1) : 

        has_changed = True
        while(has_changed):
            has_changed = False
            temp_result = []
            list_indice = []
            new_list_score = []
            for i in range(len(list_point)):
                Added_new_element = False
                for j in range(i+1, len(list_point)):
                    if (math.fabs(list_point[i][0] - list_point[j][0]) <= 3*step_x and
                        math.fabs(list_point[i][1] - list_point[j][1]) <= 3*step_y and j not in list_indice):

                        index_max_score = list_score.index(max([list_score[i],list_score[j]]))
                        temp_result.append((list_point[index_max_score][0], list_point[index_max_score][1]))
                        new_list_score.append(max([list_score[i],list_score[j]]))
                        list_indice.append(j)
                        Added_new_element = True

                if Added_new_element == False and i not in list_indice:
                    temp_result.append(list_point[i])
                    new_list_score.append(list_score[i])

                has_changed = has_changed | Added_new_element

            list_point = copy.deepcopy(temp_result)
            list_score = copy.deepcopy(new_list_score)

        return list_point, new_list_score
    
    elif(mode == 2) :
      
        pos_max_score = []

        if(list_score) :
            index_max_score = list_score.index(max(list_score))
            pos_max_score.append((list_point[index_max_score][0], list_point[index_max_score][1]))
            
        return pos_max_score
    
    
    

In [ ]:
##
# Search visages on the gray image.
# Return positions, images and scores of positives results.
##
def search_visage(gray_image, size_filter_X, size_filter_Y, model, threshold):
    
    # Raise an exception, if we can't apply the fitlter
    width, height = gray_image.shape[0], gray_image.shape[1]
    
    if width < size_filter_Y and height < size_filter_X :
        raise Exception ("impossible to crop properly")
        
    if (size_filter_X/2) % 2 != 0 or (size_filter_Y/2) % 2 != 0:
        raise Exception ("All dimension of the filter should be pair")
    
    # Loop on the image.
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    step_x = int(fil_divi_2_X/3)
    step_y = int(fil_divi_2_Y/3)
    result = []
    listImgPos = []
    listScorePos = []
    for y in range(fil_divi_2_X, width - fil_divi_2_X, step_x):
        for x in range(fil_divi_2_Y, height - fil_divi_2_Y, step_y):
            crop_img = gray_image[y - fil_divi_2_Y: y + fil_divi_2_Y, x - fil_divi_2_X : x + fil_divi_2_X]
            
            crop_imag_copy  = copy.deepcopy(crop_img)
            
            crop_img = np.array(crop_img)
            crop_img.resize((1,36,36,1))
            
            score = model.predict(crop_img,verbose = 0)[0]
            
            if int(score + threshold) == 1:
                result.append((x,y))
                listScorePos.append(score)
                listImgPos.append(crop_imag_copy)
    
    result, listScorePos = compute_highest_score(result, listScorePos, step_x, step_y, 1)
    return result, listImgPos, listScorePos


In [ ]:
##
# Draw a rectangle on the image (img)
# cor_x, cor_y : Center's point 
# size_filter_X, size_filter_Y : Width and height
##
def draw_rectangle(img, cor_x, cor_y, size_filter_X, size_filter_Y ):
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    cv2.rectangle(img, (cor_x - fil_divi_2_X, cor_y - fil_divi_2_Y), (cor_x + fil_divi_2_X, cor_y + fil_divi_2_Y), (0,255,0), 2)
    return img

In [ ]:
##
# Pyramid algorithm : First version. 
# Deprecated, use pyramid_search_all instead.
##
def pyramid_search (path_image, size_filter_X, size_filter_Y, nb_iterations, model, threshold):
    
    # We get and tranform the image in gray.
    img_search = cv2.imread(path_image)
    gray_image = cv2.cvtColor(img_search, cv2.COLOR_BGR2GRAY)
    gray_imag_rect  = copy.deepcopy(gray_image)
    
    # Make the reduction.
    height, width = gray_image.shape
    nbPixelToReduceX = int(int(width  - size_filter_X) / (nb_iterations-1))
    nbPixelToReduceY = int(int(height - size_filter_Y) / (nb_iterations-1))
    
    results = []
    for ite in range(nb_iterations) :
        
        height, width = gray_image.shape
        
        if(ite == nb_iterations-1):
            gray_image = cv2.resize(gray_image, (size_filter_X, size_filter_Y)) # Last iteration
            size_filter_X_new = width - nbPixelToReduceX
            size_filter_Y_new = height - nbPixelToReduceY
            height, width = gray_image.shape
        elif(ite == 0):
            size_filter_X_new = size_filter_X - nbPixelToReduceX
            size_filter_Y_new = size_filter_Y - nbPixelToReduceY
        else :
            gray_image = cv2.resize(gray_image, (width - nbPixelToReduceX, height - nbPixelToReduceY))   
        
        gray_imag_rect_copy  = copy.deepcopy(gray_image)
            
        # We get all square that seems to be a visage.
        result, listImgPos, listScorePos = search_visage(gray_image, size_filter_X, size_filter_Y, model, threshold)
        
        size_filter_X_new = nbPixelToReduceX + size_filter_X_new
        size_filter_Y_new = nbPixelToReduceY + size_filter_Y_new
        
        results += result
        
        # Draw red rectangle.
        for x, y in result:
            print('(x, y) = (', x, ', ', y, ') on image : ', gray_image.shape )
            gray_imag_rect = draw_rectangle(gray_image, x, y, size_filter_X, size_filter_Y)
        
        
        ## Display positives images
        #for l in listImgPos :
        #    plt.figure()
        #    plt.imshow(l)
            
        ## Display initial image with reds rectangles. 
        if(result == []) :
            plt.figure()
            plt.imshow(gray_image)
        else :
            plt.figure()
            plt.imshow(gray_imag_rect)
        
        # Remove rectangle before next iteration.
        gray_image = gray_imag_rect_copy       
        
    return gray_imag_rect

In [ ]:
##
# Pyramid algorithm : 
# Reduce image's size at each iteration until 36x36. 
# Filter keep the size of 36x36. 
##
def pyramid_search_all (path_image, size_filter_X, size_filter_Y, nb_iterations, model, threshold):
    print('Searching faces on different scales of the image ...')
    # We get and tranform the image in gray.
    img_search = cv2.imread(path_image)
    gray_image = cv2.cvtColor(img_search, cv2.COLOR_BGR2GRAY)
    gray_imag_rect  = copy.deepcopy(gray_image)
    orig_imag = copy.deepcopy(gray_image)
    origin_height, origin_width = orig_imag.shape
    
    # Make the reduction.
    height, width = gray_image.shape
    nbPixelToReduceX = int(int(width  - size_filter_X) / (nb_iterations-1))
    nbPixelToReduceY = int(int(height - size_filter_Y) / (nb_iterations-1))
    
    results = []
    list_Score_Pos_Total = []
    for ite in range(nb_iterations) :
        
        height, width = gray_image.shape
        
        if(ite == nb_iterations-1):
            gray_image = cv2.resize(gray_image, (size_filter_X, size_filter_Y)) # Last iteration
            size_filter_X_new = width - nbPixelToReduceX
            size_filter_Y_new = height - nbPixelToReduceY
            height, width = gray_image.shape
        elif(ite == 0):
            size_filter_X_new = size_filter_X - nbPixelToReduceX
            size_filter_Y_new = size_filter_Y - nbPixelToReduceY
        else :
            gray_image = cv2.resize(gray_image, (width - nbPixelToReduceX, height - nbPixelToReduceY))   
        
        gray_imag_rect_copy  = copy.deepcopy(gray_image)
            
        # We get all square that seems to be a visage.
        result, listImgPos, list_Score_Pos = search_visage(gray_image, size_filter_X, size_filter_Y, model, threshold)
        
        size_filter_X_new = nbPixelToReduceX + size_filter_X_new
        size_filter_Y_new = nbPixelToReduceY + size_filter_Y_new
        
        nh, nw = gray_image.shape
        w = origin_width/nw
        h = origin_height/nh
    
        
        # Keep results in lists.
        for x, y in result:
            results.append((int(x*w), int(y*h), int(size_filter_X*w), int(size_filter_Y*h)))
        for i in range(len(list_Score_Pos)):
            list_Score_Pos_Total.append(list_Score_Pos[i])

    return orig_imag, results, list_Score_Pos_Total

In [ ]:
##
# Draw all rectangles (results) on the image (orig_imag)
##
def draw_rectangles_all(orig_imag, results) :
    # Draw red rectangle.
    for x, y, w, h in results:
        gray_imag_rect = draw_rectangle(orig_imag, x, y, w, h)

    ## Display initial image with reds rectangles. 
    if(results == []) :
        plt.figure()
        plt.imshow(orig_imag)
    else :
        plt.figure()
        plt.imshow(gray_imag_rect)
   

In [ ]:
##
# Search the best rectangles on image. 
# Use clustering (DBSCAN), and choose the rectangle 
# with the highest score of each cluster.
##
def merge_rect(rectangles, list_score, img_w, img_h) :
    X =[]
    nb = len(rectangles)
    for x, y, w, h in rectangles :
        X.append([x, y])
    eps = max([max(x[3],x[2]) for x in rectangles])/3
    print('eps DBSCAN = ', eps)
    db = DBSCAN(eps=eps, min_samples=5, metric='euclidean').fit(X)
    
    labels = db.labels_

    clusters = {}
    score_clusters = {}
    for i in range(0, len(labels)) :
        
        num_cluster = labels[i]
        if num_cluster != -1 : # -1 = Noise
            if num_cluster in clusters :
                
                # Take the widest rectangle
                #if rectangles[i][2] > clusters[labels[i]][2] :
                #    clusters[labels[i]] = rectangles[i]
                
                # Take the best score
                if(list_score[i] > score_clusters[labels[i]]) :
                    clusters[labels[i]]       = rectangles[i]
                    score_clusters[labels[i]] = list_score[i]
              
            else :
                clusters[num_cluster]       = rectangles[i]
                score_clusters[num_cluster] = list_score[i]
                
    results = []
    for c in clusters :
        results.append(clusters[c])
    return results
    

In [ ]:
model = load_model_keras('model_hard_train_1.h5')

In [ ]:
for path in listdir('./img/') :
    img_org, res, listScorePos = pyramid_search_all('./img/' + path, SIZE_FILTER_X, SIZE_FILTER_Y, NB_ITERATION, model, THRESHOLD_MODEL)
    img_h, img_w = img_org.shape
    img = copy.deepcopy(img_org)
    #print('Before merging the rectangles')
    draw_rectangles_all(img, res)
    results = merge_rect(res, listScorePos, img_w, img_h)
    img_c = copy.deepcopy(img_org)
    #print('After merging the rectangles')
    draw_rectangles_all(img_c, results)    

<h1>HARD Example</h1>

In [ ]:
def pyramid_search_difficult (path_image, size_filter_X, size_filter_Y, nb_iterations, model, threshold):
    # We get and tranform the image in gray.
    img_search = cv2.imread(path_image)
    gray_image = cv2.cvtColor(img_search, cv2.COLOR_BGR2GRAY)
    gray_imag_rect  = copy.deepcopy(gray_image)
    
    # Make the reduction.
    # --> search
    height, width = gray_image.shape
    nbPixelToReduceX = int((width  - size_filter_X) / (nb_iterations -1) )
    nbPixelToReduceY = int((height - size_filter_Y) / (nb_iterations -1) ) 
    
    list_reponse = []
    for ite in range(nb_iterations) :
        
        height, width = gray_image.shape
        
        if(ite == nb_iterations-1):
            gray_image = cv2.resize(gray_image, (size_filter_X, size_filter_Y)) # Last iteration
            size_filter_X_new = width - nbPixelToReduceX
            size_filter_Y_new = height - nbPixelToReduceY
            height, width = gray_image.shape
        elif(ite == 0):
            size_filter_X_new = size_filter_X - nbPixelToReduceX
            size_filter_Y_new = size_filter_Y - nbPixelToReduceY
        else :
            gray_image = cv2.resize(gray_image, (width - nbPixelToReduceX, height - nbPixelToReduceY))   
        
        gray_imag_rect_copy  = copy.deepcopy(gray_image)
        
        # we get a list of all visage.
        list_reponse = list_reponse + search_visage(gray_image, size_filter_X, size_filter_Y, model, threshold)
        
    return list_reponse

In [ ]:
def search_visage(gray_image, size_filter_X, size_filter_Y, model, threshold):
    
    # Raise an exception, if we can't apply the fitlter
    width, height = gray_image.shape[0], gray_image.shape[1]
    
    if width < size_filter_Y and height < size_filter_X :
        raise Exception ("impossible to crop properly")
        
    if (size_filter_X/2) % 2 != 0 or (size_filter_Y/2) % 2 != 0:
        raise Exception ("All dimension of the filter should be pair")
    
    # loop on the image.
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    step_x = int(fil_divi_2_X/3)
    step_y = int(fil_divi_2_Y/3)
    listImgPos = []
    for y in range(fil_divi_2_X, height - fil_divi_2_X, step_x):
        for x in range(fil_divi_2_Y, width - fil_divi_2_Y, step_y):
            crop_img = gray_image[y - fil_divi_2_Y: y + fil_divi_2_Y, x - fil_divi_2_X : x + fil_divi_2_X]
            
            crop_imag_copy  = copy.deepcopy(crop_img)
            crop_img = np.array(crop_img)
            crop_img.resize((1,36,36,1))
            
            if int(model.predict(crop_img,verbose = 0)[0] + threshold) == 1:
                listImgPos.append(crop_imag_copy)

    return listImgPos

In [ ]:

path_repo = "data_difficult_faces/0/"
path_save = "data_save_difficult_no_faces/"
def find_and_save_difficult_images(path_repo, path_save, threshold, model, indice):
    list_image = []
    # get the image.
    for index, filename in enumerate(os.listdir(path_repo)):
        if filename == ".DS_Store":
            continue
        list_difficult_faces =  pyramid_search_difficult(os.path.join(path_repo, filename), SIZE_FILTER_X, SIZE_FILTER_Y, NB_ITERATION, model, threshold)
        print("Traite l'image : "+ str(index) + " find : "+ str(len (list_difficult_faces)))
        list_image +=  list_difficult_faces
    
    max_indice = 0    
    for index, image in enumerate(list_image):
        width, height = image.shape[0], image.shape[1]
        max_indice = indice + index
        if width == 36 and height == 36:
            cv2.imwrite(os.path.join(path_save, str(max_indice) + ".jpg"), image) 
            print()
    
    return max_indice

In [ ]:
THRESHOLD = 0.7
def update_model (path_save, threshold):
    path_repo = "data_difficult_faces/"
    indice = 0
    threshold = 0.2
    model = None
    
    lenght_repo = 0
    if ".DS_Store" in os.listdir("data_difficult_faces/"):
        lenght_repo = len(os.listdir("data_difficult_faces/"))-1
    else:
        lenght_repo = len(os.listdir("data_difficult_faces/"))-1
        
    for ite in range(lenght_repo):
        print ("iteration : " + str(ite) + " threshold : "+  str(1-threshold))
        cur_path_rep = path_repo + str(ite) + "/"
        print(cur_path_rep)
        model = create_model_with_hard_example.get_model(1)
        indice += find_and_save_difficult_images(cur_path_rep, path_save, threshold, model, indice)
        threshold = threshold * THRESHOLD
            model.save('model_it'+ str(ite) +''.h5')
    

    model.save('my_model.h5')

In [ ]:
update_model(path_save, 0.2)